# Analyse de la popularité des recettes

In [2]:
from pathlib import Path
import importlib.util

import ipynbname # permet de trouver le chemin du notebook en cours
REPO_ROOT = ipynbname.path().parents[2]  # dataset_analysis/popularity -> dataset_analysis -> racine
print("Racine détectée:", REPO_ROOT)

script_path = REPO_ROOT / "scripts" / "download_data.py"
spec = importlib.util.spec_from_file_location("download_data", script_path)
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)

mod.download_and_extract()  # télécharge dans <repo>/data/


Racine détectée: /Users/gregoirepetit/mangetamain
Racine projet : /Users/gregoirepetit/mangetamain
Téléchargement vers : /Users/gregoirepetit/mangetamain/data
Dataset URL: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
✅ Téléchargement terminé.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Import des datasets
data_path = REPO_ROOT / "data"
recipes = pd.read_csv(data_path / "RAW_recipes.csv")
interactions = pd.read_csv(data_path / "RAW_interactions.csv")

## Exploration initiale des données

Commençons par comprendre la structure des données et identifier les métriques de popularité disponibles.

In [ ]:
recipes.head() # Afficher les premières lignes du DataFrame des recettes

In [ ]:
interactions.head() # Afficher les premières lignes du DataFrame des interactions

- On peut repérer quelques colonnes intéressantes dans les datasets :
  - Dans `RAW_recipes.csv` : `id`, `minutes`, `submitted`, `n_steps`, `n_ingredients`, `contributor_id`
  - Dans `RAW_interactions.csv` : `user_id`, `recipe_id`, `rating`, `date`, `review`.

In [ ]:
# On contrôle les valeurs manquantes
print("Valeurs manquantes dans les recettes par variable: \n", recipes.isna().sum(), "\n")
print("Valeurs manquantes dans les interactions par variable: \n", interactions.isna().sum())

In [ ]:
# On trace des boxplots pour visualiser la distribution de certaines variables numériques
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.boxplot(y=recipes['minutes'], ax=axes[0])
axes[0].set_title('Temps de préparation (minutes)')
axes[0].set_ylabel('minutes')

sns.boxplot(y=recipes['n_steps'], ax=axes[1])
axes[1].set_title("Nombre d'étapes")
axes[1].set_ylabel('n_steps')

sns.boxplot(y=recipes['n_ingredients'], ax=axes[2])
axes[2].set_title("Nombre d'ingrédients")
axes[2].set_ylabel('n_ingredients')

plt.tight_layout()
plt.show()

- Les distributions du temps de préparation (`minutes`), du nombre d'étapes (`n_steps`) et du nombre d'ingrédients (`n_ingredients`) sont toutes très asymétriques, avec une majorité de recettes ayant des valeurs faibles et quelques recettes extrêmes tirant les distributions vers le haut.
- On observe la présence de nombreux outliers, en particulier pour le temps de préparation, ce qui indique que certaines recettes demandent beaucoup plus de temps que la moyenne.
- La plupart des recettes nécessitent un nombre modéré d'étapes et d'ingrédients, mais il existe des recettes très complexes avec de nombreux ingrédients ou étapes.
- Ces variables présentent donc une forte hétérogénéité, ce qui devra être pris en compte dans l’analyse de la popularité des recettes.

### Traitement : 'minutes'

In [ ]:
# Afficher la recette avec le temps de préparation minimum
print("Recette avec le temps de préparation minimum :")
recipes.loc[[recipes['minutes'].idxmin()]]

In [ ]:
# Afficher la recette avec le temps de préparation maximum
print("Recette avec le temps de préparation maximum :")
recipes.loc[[recipes['minutes'].idxmax()]]  # Double brackets = DataFrame

- On constate un temps anormalement grand pour le temps de préparation maximum&nbsp;:
  
  > 2&nbsp;147&nbsp;483&nbsp;647 minutes  
  > soit environ `4085.8 ans` (`2147483647 / (60 × 24 × 365)`), ce qui est clairement une erreur ou une valeur aberrante.

- Nous faisons le choix de retirer cette valeur extrême de notre analyse pour éviter qu'elle n'influence les résultats.
- On remarque également que certaines recettes ont un temps de préparation de 0 minutes, ce qui est probablement une erreur ou une valeur manquante mal codée. Nous allons les traiter en les remplaçant par des valeurs NA.

In [ ]:
# Retirer la recette avec le temps de préparation maximum (valeur aberrante)
recipes = recipes[recipes['minutes'] != recipes['minutes'].max()]

# Indiquer NA pour celles qui ont 0 minutes
recipes.loc[recipes['minutes'] == 0, 'minutes'] = pd.NA

In [ ]:
# Nouveau boxplot pour constater la distribution après le traitement
sns.boxplot(x = 'minutes', data = recipes)
plt.title("Boxplot du temps de préparation (minutes)")
plt.xlabel("minutes")
plt.show()

In [ ]:
# On affiche les 5 recettes avec le plus grand temps de préparation (minutes)
recipes.sort_values('minutes', ascending=False).head(5)

- La recette "how to preserve a husband" semble être une plaisanterie ou une erreur de saisie plutôt qu'une vraie recette, son temps de préparation étant aussi significativement au-dessus des autres sans élément concret pour le justifier. 
  - Nous ferons le choix de la retirer pour la suite de l'analyse.
- Il semblerait que que le reste des recettes correspondent à des fermentations, d'où leur durée très grande.


In [ ]:
# On retire la recette "how to preserve a husband"
recipes = recipes[recipes['minutes'] != recipes['minutes'].max()]

In [ ]:
# Calculs des statistiques descriptives de la colonne 'minutes'
stats_minutes = recipes['minutes'].describe()
stats_minutes['variance'] = recipes['minutes'].var()
stats_minutes['skewness'] = recipes['minutes'].skew()
stats_minutes['kurtosis'] = recipes['minutes'].kurtosis()
stats_minutes

- Les statistiques descriptives de la colonne `minutes` montrent une forte asymétrie :
    - La médiane (50%) est de 40 minutes, alors que la moyenne est beaucoup plus élevée (~123,7 minutes), ce qui indique la présence de valeurs extrêmes tirant la moyenne vers le haut.
    - L'écart-type est très grand (1982 minutes), bien supérieur à la médiane, ce qui confirme la dispersion importante des temps de préparation.
    - Le minimum est de 1 minute, le maximum de 288 000 minutes (soit 200 jours), ce qui révèle des valeurs aberrantes ou des recettes très particulières (fermentations, erreurs de saisie).
    - Le skewness (asymétrie) et la kurtosis (aplatissement) sont extrêmement élevés, ce qui traduit une distribution très éloignée de la normale, avec de nombreux outliers.

- Passer en échelle logarithmique permet de mieux visualiser la distribution des temps de préparation :
    - L'échelle linéaire "écrase" la majorité des recettes (à faible durée) à cause de quelques valeurs extrêmes.
    - L'échelle logarithmique "compresse" les grandes valeurs et "étale" les petites, rendant la distribution plus lisible et permettant d'observer les tendances pour la majorité des recettes, tout en conservant l'information sur les valeurs élevées.

In [ ]:
ax = sns.boxplot(x='minutes', data=recipes)
ax.set_xscale('log')
ax.set_title("Boxplot du temps de préparation (minutes) (log)")
ax.set_xlabel("minutes (log)")
plt.show()

- On remarque encore une présence élevée de valeurs extrêmes, malgré l'élimination des recettes aberrantes, l'ajout des NA pour les recettes à 0 et le passage en échelle logarithmique.
- Cela suggère que le temps de préparation des recettes est intrinsèquement très variable, avec une majorité de recettes rapides et quelques recettes très longues.
- Nous pourrions envisager d'autres transformations ou segmentations pour mieux capturer cette variabilité dans l'analyse de la popularité des recettes.

### Traitement : 'n_steps'

In [ ]:
sns.boxplot(x='n_steps', data=recipes)
plt.title("Boxplot du nombre d'étapes")
plt.xlabel("n_steps")
plt.show()

- Le boxplot du nombre d'étapes (`n_steps`) montre une distribution très asymétrique, avec une majorité de recettes ayant peu d'étapes et une longue "queue" de valeurs élevées.
- On observe de nombreux outliers au-dessus de la borne supérieure du boxplot, ce qui indique la présence de recettes particulièrement complexes ou de valeurs aberrantes.

In [ ]:
recipes['n_steps'].describe()

- La moyenne de la colonne n_steps étant aux alentours de 10 étapes et le maximum à 145 avec un écart-type de 6, nous pouvons proposer de concaténer le n_steps en catégories :
    - Simple (1-5 étapes)
    - Modéré (6-10 étapes)
    - Complexe (11-20 étapes)
    - Très complexe (>20 étapes)

In [ ]:
# Catégorisation simple pour l'analyse
def categorize_complexity(n_steps):
    if n_steps <= 5: return "Simple"
    elif n_steps <= 10: return "Modéré" 
    elif n_steps <= 20: return "Complexe"
    else: return "Très complexe"

recipes['complexity'] = recipes['n_steps'].apply(categorize_complexity)

# Distribution par complexité
complexity_dist = recipes['complexity'].value_counts()
print(f"\n Distribution par complexité:")
for category, count in complexity_dist.items():
    pct = count/len(recipes)*100
    print(f"• {category}: {count:,} recettes ({pct:.1f}%)")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Histogramme avec échelle logarithmique sur l'axe Y (pour voir les queues)
sns.histplot(data=recipes, x='n_steps', bins=50, kde=False, ax=ax1, alpha=0.7, color='skyblue')
ax1.axvline(recipes['n_steps'].mean(), color='red', linestyle='--', alpha=0.8, label=f'Moyenne: {recipes["n_steps"].mean():.1f}')
ax1.axvline(recipes['n_steps'].median(), color='orange', linestyle='--', alpha=0.8, label=f'Médiane: {recipes["n_steps"].median():.0f}')
ax1.set_title('Distribution du nombre d\'étapes (échelle log)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Nombre d\'étapes')
ax1.set_ylabel('Fréquence (log)')
ax1.set_yscale('log')  # Échelle logarithmique pour mieux voir les queues
ax1.legend()
ax1.set_xlim(0, 100)
ax1.grid(True, alpha=0.3)

# 2. Boxplot uniquement pour la catégorie "Très complexe"
sns.boxplot(data=recipes[recipes['complexity'] == "Très complexe"], x='complexity', y='n_steps', ax=ax2, palette='viridis')
ax2.set_title('Distribution pour "Très complexe"', fontsize=14, fontweight='bold')
ax2.set_xlabel('Niveau de complexité')
ax2.set_ylabel('Nombre d\'étapes')
ax2.set_ylim(0, 80)

plt.tight_layout()
plt.show()


- Utilisation de l'échelle log verticale pour l'histogramme du nombre d'étapes pour mieux visualiser la queue de la distribution et les outliers.
- Boxplot uniquement pour la catégorie "Très complexe" pour mieux visualiser les statistiques descriptives et la présence d'outliers dans cette catégorie.



In [ ]:
recipes[recipes['n_steps'] >= 40]

- ajout d'un tri avec n_steps >= 40 & minutes >= 1200 (20 heures) pour repérer les recettes les plus longues et complexes, potentiellement des erreurs ou des recettes très particulières (fermentations, etc.).

### Traitement : 'n_ingredients'

## Traitement : Interactions

In [ ]:
interactions.head()

### Traitement : 'rating'

In [ ]:
plt.figure(figsize=(10, 6))
bins = np.arange(0, 6, 1)  # De 0 à 5 par pas de 1
sns.histplot(interactions['rating'], bins=bins, kde=False, color='skyblue', alpha=0.7)
plt.title("Distribution des notes (rating) - Échelle logarithmique", fontsize=14, fontweight='bold')
plt.xlabel("Note")
plt.ylabel("Nombre d'interactions (log)")
plt.yscale('log')  # Échelle logarithmique pour mieux voir les détails
plt.xticks(range(int(interactions['rating'].min()), int(interactions['rating'].max()) + 1))

plt.show()

- on regarde les notes à 0 :

In [ ]:
interactions[interactions['rating'] == 0]

- Il semblerait que les notes à 0 soient simplement des commentaires sans évaluation chiffrée, ou des évaluations non renseignées.


In [ ]:
# Calcul du nombre et du pourcentage d'interactions avec rating = 0
nb_rating_0 = (interactions['rating'] == 0).sum()
pct_rating_0 = nb_rating_0 / len(interactions) * 100
print(f"Nombre d'interactions avec rating = 0 : {nb_rating_0:,} ({pct_rating_0:.2f}%)")

- Nous faisons le choix de les passer à NA pour ne pas biaiser l'analyse des notes réelles (1 à 5).
- Elles constituent une part non négligeable des intéractions, ce qui suggère que de nombreux utilisateurs laissent des commentaires sans évaluer la recette.

In [ ]:
# Indiquer NA pour les ratings = 0
interactions.loc[interactions['rating'] == 0, 'rating'] = pd.NA

### Traitement : 'review'

In [ ]:
# Afficher les lignes sans commentaire textuel
interactions[interactions['review'].isna()].head()

In [ ]:
# Calcul du nombre et du pourcentage d'interactions sans commentaire textuel
nb_no_review = interactions['review'].isna().sum()
pct_no_review = nb_no_review / len(interactions) * 100
print(f"Nombre d'interactions sans commentaire textuel : {nb_no_review:,} ({pct_no_review:.2f}%)")

# Calcul de la note moyenne pour les interactions sans commentaire textuel
mean_rating_no_review = interactions.loc[interactions['review'].isna(), 'rating'].mean()
print(f"Note moyenne des interactions sans commentaire textuel : {mean_rating_no_review:.2f}")

In [ ]:
# Analyse de la longueur des commentaires
interactions['review_length'] = interactions['review'].str.len()

# Statistiques descriptives sur la longueur des reviews
interactions['review_length'].describe()

In [ ]:
# Calculer la longueur moyenne des reviews par rating
mean_review_length_by_rating = interactions.groupby('rating')['review_length'].mean()

plt.figure(figsize=(8, 5))
mean_review_length_by_rating.plot(kind='bar', color='skyblue')
plt.title("Longueur moyenne des reviews par note")
plt.xlabel("Note")
plt.ylabel("Longueur moyenne du commentaire")
plt.show()


## Préparation des données pour l'analyse univariée de la popularité

### Agrégations par recette

In [ ]:
recipe_metrics = interactions.groupby('recipe_id').agg({
    'rating': ['count', 'mean', 'median', 'std'],
    'user_id': 'nunique', # Nombre d'utilisateurs uniques ayant noté la recette
    'review': lambda x: x.notna().sum() # Nombre de reviews avec texte
}).round(3) # Arrondir à 3 décimales

# Applatir les colonnes multi-niveaux
recipe_metrics.columns = ['n_interactions', 'avg_rating', 'median_rating', 'std_rating', 'n_unique_users', 'n_reviews_text'] # n_ratings --> n_interactions pour plus de clarté

In [ ]:
recipe_metrics.head(10)

### Ajout de métriques robustes de popularité : Bayésienne + Wilson

In [ ]:
# Bayesian Average Rating (BAR)
global_mean = interactions['rating'].mean()
prior_count = 10  # Nombre de "votes" fictifs = Paramètre bayésien

# Calcul du BAR pour chaque recette
recipe_metrics['bayes_mean'] = (
    (recipe_metrics['avg_rating'] * recipe_metrics['n_interactions'] + global_mean * prior_count) /
    (recipe_metrics['n_interactions'] + prior_count)
).round(3)

- L’ajout de la moyenne bayésienne (Bayesian Average Rating) permet de corriger le biais lié au faible nombre d’interactions pour certaines recettes. Contrairement à la moyenne classique, cette approche intègre une "prior" globale et réduit l’impact des recettes peu notées qui pourraient apparaître artificiellement en tête du classement. 
- Cela garantit une comparaison plus robuste et équitable entre toutes les recettes, en tenant compte à la fois de la qualité perçue et de la quantité d’avis disponibles. 
- Cette statistique sera donc privilégiée pour l’analyse de la popularité afin d’éviter les effets de bord dus aux extrêmes ou aux recettes peu évaluées.

In [ ]:
# Wilson Lower Bound (WLB) pour les notes binaires (4-5 étoiles = positif)
def wilson_lower_bound(positives, total, confidence=0.95):
    if total == 0:
        return 0
    p = positives / total
    z = 1.96 if confidence == 0.95 else 1.64  # z-score
    denominator = 1 + z**2 / total
    center = (p + z**2 / (2 * total)) / denominator
    margin = z * np.sqrt((p * (1 - p) + z**2 / (4 * total)) / total) / denominator
    return max(0, center - margin)

Le score Wilson Lower Bound (WLB) est une méthode statistique permettant d’estimer une borne inférieure de confiance pour la proportion de notes positives (ici, les notes ≥ 4) d’une recette, en tenant compte du nombre total de votes et de la variabilité. 

Contrairement à une simple proportion, le WLB pénalise les recettes avec peu de votes et donne une estimation plus prudente de la popularité réelle. Il est particulièrement utile pour éviter que des recettes peu évaluées mais avec uniquement des avis positifs ne soient surclassées dans le classement.

La formule utilisée est :

$$
WLB = \frac{p + \frac{z^2}{2n} - z \sqrt{\frac{p(1-p) + \frac{z^2}{4n}}{n}}}{1 + \frac{z^2}{n}}
$$

où :
- $p$ = proportion de notes positives (≥ 4)
- $n$ = nombre total de notes
- $z$ = score de confiance (1.96 pour 95%)

Ce score représente la borne inférieure de l’intervalle de confiance à 95% pour la proportion de notes positives. Plus le nombre de votes est élevé, plus la borne est proche de la proportion réelle. Pour les recettes peu notées, le score est fortement réduit, ce qui limite les faux positifs dans le classement de popularité.

In [ ]:
# Calculer Wilson LB
ratings_by_recipe = interactions.groupby('recipe_id')['rating'].apply(list).to_dict()
wilson_scores = {}
for recipe_id, ratings in ratings_by_recipe.items():
    ratings_clean = [r for r in ratings if pd.notna(r)]
    if ratings_clean:
        positives = sum(1 for r in ratings_clean if r >= 4)
        total = len(ratings_clean)
        wilson_scores[recipe_id] = wilson_lower_bound(positives, total)
    else:
        wilson_scores[recipe_id] = 0

recipe_metrics['wilson_lb'] = recipe_metrics.index.map(wilson_scores)

### Jointure avec Recipes + métriques temporelles

In [ ]:
# Convertir 'submitted' en datetime
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
current_date = pd.Timestamp.now()  # Date de référence pour le calcul de l'ancienneté
recipes['age_months'] = ((current_date - recipes['submitted']).dt.days / 30.44).round(1)

In [ ]:
# Jointure principale
df_analysis = recipes.merge(recipe_metrics, left_on='id', right_index=True, how='inner')

- Choix d'une jointure 'inner' pour ne conserver que les recettes ayant au moins une interaction, ce qui correspond à notre problématique d'analyse de la popularité en fonction de l'effort culinaire.

In [ ]:
# Métriques d'engagement normalisées par le temps
df_analysis['interactions_per_month'] = df_analysis['n_interactions'] / np.maximum(1, df_analysis['age_months'])

In [ ]:
# Log transformation pour minutes (après avoir retiré les 0 et valeurs aberrantes)
df_analysis = df_analysis[df_analysis['minutes'].notna() & (df_analysis['minutes'] > 0)]
df_analysis['log_minutes'] = np.log(df_analysis['minutes'])

In [ ]:
df_analysis.head()

In [ ]:
# Statistiques descriptives des métriques de popularité
popularity_cols = ['n_interactions', 'avg_rating', 'bayes_mean', 'wilson_lb', 'interactions_per_month']
df_analysis[popularity_cols].describe().round(3)

## 1) Analyse univariée de la popularité

### 1.1) Visualisation des distributions des métriques de popularité

In [ ]:
# Calcul du nombre optimal de bins avec la règle de Freedman-Diaconis
    # bins = (max - min) / (2*IQR/n^(1/3))
def freedman_diaconis_bins(data):
        data_clean = data.dropna()
        n = len(data_clean)
        q75, q25 = np.percentile(data_clean, [75, 25])
        iqr = q75 - q25
        if iqr > 0:
            bin_width = 2 * iqr / (n ** (1/3))
            bins = int(np.ceil((data_clean.max() - data_clean.min()) / bin_width))
        else:
            bins = int(np.ceil(np.sqrt(n)))  # Fallback
            
        # Limiter entre 10 et 100 bins
        return max(10, min(100, bins))

Cette fonction utilitaire calcule le nombre de classes optimal pour nos histogrammes en appliquant la règle de Freedman-Diaconis :
- on ignore les valeurs manquantes pour ne garder que les observations valides ;
- on calcule l'écart interquartile (IQR) afin de mesurer la dispersion réelle de la distribution ;
- on en déduit une largeur de classe puis on borne le nombre de classes entre 10 et 100 pour garder des graphiques lisibles ;
- si les données sont trop concentrées (IQR nul), on retombe sur une règle de secours basée sur √n.
Cela harmonise la granularité des histogrammes entre toutes les métriques de popularité.

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 15))

# 1- Satisfaction: Qualité perçue via les ratings
satisfaction_vars = ['avg_rating', 'bayes_mean', 'wilson_lb']

for i, var in enumerate(satisfaction_vars):
    optimal_bins = freedman_diaconis_bins(df_analysis[var])
    # Histogramme
    sns.histplot(df_analysis[var], bins = optimal_bins, kde=True, ax=axes[0, i], alpha=0.7)
    axes[0, i].set_title(f'Distribution: {var}', fontweight='bold')
    axes[0, i].axvline(df_analysis[var].mean(), color='red', linestyle='--', 
                       label=f'Moyenne: {df_analysis[var].mean():.3f}')
    axes[0, i].axvline(df_analysis[var].median(), color='orange', linestyle='--',
                       label=f'Médiane: {df_analysis[var].median():.3f}')
    axes[0, i].legend()
    
    # Statistiques
    skew = df_analysis[var].skew()
    kurt = df_analysis[var].kurtosis()
    axes[0, i].text(0.7, 0.9, f'Skew: {skew:.2f}\nKurt: {kurt:.2f}', 
                    transform=axes[0, i].transAxes, bbox=dict(boxstyle="round", facecolor='wheat'))

# 2- Engagement: popularité via le volume/rythme des interactions
engagement_vars = ['n_interactions', 'interactions_per_month', 'n_unique_users']

for i, var in enumerate(engagement_vars):
    optimal_bins = freedman_diaconis_bins(df_analysis[var])
    # Vérifier si très asymétrique
    is_skewed = df_analysis[var].skew() > 2
    
    # Histogramme
    sns.histplot(df_analysis[var], bins = optimal_bins, kde=False, ax=axes[1, i], alpha=0.7)
    
    # Appliquer échelle log AVANT les lignes verticales et labels
    if is_skewed:
        axes[1, i].set_yscale('log')
        axes[1, i].set_ylabel('Fréquence (log)')
        axes[1, i].set_title(f'{var} (échelle log Y)', fontweight='bold')
    else:
        axes[1, i].set_ylabel('Fréquence')
        axes[1, i].set_title(f'Distribution: {var}', fontweight='bold')
    
    # Lignes verticales APRÈS l'échelle log
    axes[1, i].axvline(df_analysis[var].mean(), color='red', linestyle='--', 
                       label=f'Moy: {df_analysis[var].mean():.1f}')
    axes[1, i].axvline(df_analysis[var].median(), color='orange', linestyle='--',
                       label=f'Méd: {df_analysis[var].median():.1f}')
    axes[1, i].legend()
    axes[1, i].grid(True, alpha=0.3)
    
    # Boxplot correspondant
    sns.boxplot(y=df_analysis[var], ax=axes[2, i])
    if is_skewed:
        axes[2, i].set_yscale('log')
        axes[2, i].set_title(f'Boxplot: {var} (log Y)', fontweight='bold')
    else:
        axes[2, i].set_title(f'Boxplot: {var}', fontweight='bold')

plt.tight_layout()
plt.show()

**Lecture des graphiques générés :**
- Ligne 1 – `avg_rating`, `bayes_mean`, `wilson_lb` : les distributions sont concentrées autour de 4-4,5 ; les moyennes (rouge) et médianes (orange) proches montrent une satisfaction globalement élevée, avec des queues à droite pour les quelques recettes d'exception.
- Ligne 2 – `n_interactions`, `interactions_per_month`, `n_unique_users` : les histogrammes très asymétriques justifient l'échelle logarithmique sur l'axe Y, révélant une majorité de recettes peu sollicitées et une poignée de succès massifs.
- Ligne 3 – Boxplots associés : ils mettent en évidence la longue traîne des métriques d'engagement ; même en log, de nombreux points sont isolés, ce qui indique des comportements très hétérogènes entre recettes.
**À retenir :**
- Les métriques de satisfaction varient peu d'une recette à l'autre et restent proches de leurs bornes hautes.
- Les métriques d'engagement sont dominées par quelques recettes phares ; la visualisation log nous permet de comparer sans écraser le reste de la distribution.

### 1.2) Transformations avancées des données, ajout de métriques dérivées

In [ ]:
# Versions log1p des métriques d'engagement
for var in engagement_vars:
    df_analysis[f'log1p_{var}'] = np.log1p(df_analysis[var].clip(lower=0))

Cette boucle crée les versions `log1p` des métriques d'engagement en appliquant trois étapes :
- on force d'abord les valeurs négatives éventuelles à 0 via `clip(lower=0)` pour rester dans le domaine du logarithme ;
- `np.log1p` compresse ensuite la plage des valeurs tout en gardant les zéros et petits comptes lisibles ;
- on stocke enfin chaque transformation dans une nouvelle colonne `log1p_<métrique>` pour pouvoir comparer les approches.
Cette normalisation douce facilite les comparaisons futures et prépare les métriques à des modèles sensibles aux valeurs extrêmes.

In [ ]:
# Winsorization pour réduire l'impact des valeurs extrêmes
def winsorize(series, lower_pct=0.05, upper_pct=0.99):
    lower_bound = series.quantile(lower_pct)
    upper_bound = series.quantile(upper_pct)
    return series.clip(lower=lower_bound, upper=upper_bound)

for var in ["log_minutes", "log1p_n_interactions", "log1p_interactions_per_month", "log1p_n_unique_users"]:
    df_analysis[f"{var}_w"] = winsorize(df_analysis[var])

La fonction `winsorize` borne chaque série en remplaçant les valeurs trop extrêmes par les percentiles 5 % et 99 % :
- le calcul des bornes se fait sur les versions déjà transformées (log ou log1p), ce qui évite de tronquer brutalement les données brutes ;
- le clipping conserve toutes les lignes mais limite l'influence des observations aberrantes lors des analyses graphiques ou statistiques ;
- les colonnes suffixées `_w` serviront de référence plus robuste pour la suite de l'étude.

In [ ]:
# Écart à la note maximale (5) pour les métriques de satisfaction
df_analysis["rating_gap"] = 5 - df_analysis["avg_rating"]
df_analysis["bayes_gap"] = 5 - df_analysis["bayes_mean"]
df_analysis["wilson_gap"] = 1 - df_analysis["wilson_lb"]

En calculant l'écart à la note maximale pour chaque indicateur de satisfaction :
- `rating_gap` mesure combien de points séparent une recette de la note parfaite sur 5 ;
- `bayes_gap` fait de même en tenant compte de la pénalisation bayésienne appliquée aux recettes peu notées ;
- `wilson_gap` convertit le score Wilson (borne inférieure) en distance à l'optimum, ce qui homogénéise l'interprétation.
Plus l'écart est faible, plus la recette est perçue comme satisfaisante ; ces variables serviront à repérer rapidement les recettes à améliorer.

### 1.3) Distributions après transformation

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 15))

satisfaction_vars = {
    "avg_rating (gap)": "rating_gap",
    "bayes_mean (gap)": "bayes_gap",
    "wilson_lb (gap)": "wilson_gap"
}

engagement_vars = {
    "n_interactions (log1p winsor)": "log1p_n_interactions_w",
    "interactions_per_month (log1p winsor)": "log1p_interactions_per_month_w",
    "n_unique_users (log1p winsor)": "log1p_n_unique_users_w"
}

for col_idx, (title, col) in enumerate(satisfaction_vars.items()):
    data = df_analysis[col].dropna()
    bins = freedman_diaconis_bins(data)
    sns.histplot(data, bins=bins, kde=True, ax=axes[0, col_idx], alpha=0.7)
    axes[0, col_idx].set_title(f"Distribution: {title}", fontweight="bold")
    axes[0, col_idx].axvline(data.mean(), color="red", linestyle="--", label=f"Moyenne: {data.mean():.3f}")
    axes[0, col_idx].axvline(data.median(), color="orange", linestyle="--", label=f"Médiane: {data.median():.3f}")
    axes[0, col_idx].legend()
    axes[0, col_idx].text(0.70, 0.9, f"Skew: {data.skew():.2f}\nKurt: {data.kurtosis():.2f}",
                          transform=axes[0, col_idx].transAxes,
                          bbox=dict(boxstyle="round", facecolor="wheat"))

for col_idx, (title, col) in enumerate(engagement_vars.items()):
    data = df_analysis[col].dropna()
    bins = freedman_diaconis_bins(data)
    sns.histplot(data, bins=bins, kde=True, ax=axes[1, col_idx], alpha=0.7)
    axes[1, col_idx].set_ylabel("Fréquence")
    axes[1, col_idx].set_title(f"Distribution: {title}", fontweight="bold")
    axes[1, col_idx].axvline(data.mean(), color="red", linestyle="--", label=f"Moy: {data.mean():.2f}")
    axes[1, col_idx].axvline(data.median(), color="orange", linestyle="--", label=f"Méd: {data.median():.2f}")
    axes[1, col_idx].legend()
    axes[1, col_idx].grid(True, alpha=0.3)

    sns.boxplot(y=data, ax=axes[2, col_idx])
    axes[2, col_idx].set_title(f"Boxplot: {title}", fontweight="bold")

plt.tight_layout()
plt.show()


**Effets des transformations appliquées :**
- Les distributions des `rating_gap`, `bayes_gap` et `wilson_gap` sont très concentrées près de 0, confirmant que la satisfaction reste élevée et que seules quelques recettes s'éloignent sensiblement de la note maximale.
- Les versions `log1p` winsorisées des métriques d'engagement deviennent beaucoup moins étalées : le cœur de distribution apparaît clairement, ce qui rend comparables les recettes aux volumes d'interactions modestes ou intermédiaires.
- Les boxplots résiduels montrent encore des points isolés, mais ceux-ci sont désormais gérables dans des analyses corrélationnelles ou des modèles linéaires.

**Conclusion pratique :**
- Le couple `log1p` + winsorization réduit la variance et la kurtosis, stabilisant les métriques d'engagement pour les étapes analytiques qui suivent.

## 2) Analyse bivariée